In [22]:

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3
import re


In [2]:
#URL = 'https://www.bernardihondanatick.com/used-inventory/index.htm'
#URL = 'https://www.unitedautonatick.com/'

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
}

URL = 'https://www.directautomecca.com/view-inventory.aspx'
response = requests.get(URL, headers = headers)

soup = BeautifulSoup(response.text, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en-US">
 <head>
  <title>
   Boston Used cars|Used Cars Serving the Boston Area |Used Audi|Used Toyota|Used Bmw|Nissan|Honda
  </title>
  <meta content="Boston Used Cars, for best selection of used cars in Boston. Worcester Used Prices  Audis,Toyotas,Bmws,Nissans,Hondas." name="description"/>
  <meta content="Boston Used Cars, Used Audis in Boston, Used Toyotas in Boston, Used Bmw dealer in Boston, Used Nissan cars in boston, Honda certified cars in Boston." name="keywords"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type">
   <meta content="DirectAutomecca.com" name="author">
    <meta content="width=device-width, initial-scale=1" name="viewport"/>
    <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
    <meta content="Boston Used cars|Used Cars Serving the Boston Area |Used Audi|Used Toyota|Used Bmw|Nissan|Honda" name="og:title"/>
    <meta content="website" name="og:type"/>
    <meta content="DirectAutomecca.com" prope

In [126]:
# Parse a specific tag + class
def parseColumn(soup, html_tag, html_class):
    dataColumn = soup.find_all(html_tag, class_=html_class)
    new_col_list = []
    
    for i in range(0, len(dataColumn)):
        new_col_list.append(dataColumn[i].get_text())
        
    return new_col_list

# Add new column to data frame
def addColumnDF(df, arr, col_name):
    df_tmp = pd.DataFrame(arr, columns = [col_name])
    df[col_name] = df_tmp[col_name]

# Grab vehicle manufacture year
def get_valid_year(soup, html_tag, html_class):
    models = parseColumn(soup, html_tag, html_class)
    new_col_list = []
    
    for model in models:
        new_col_list.append(model[0:4])
        
    return new_col_list

# Make & Model & Vehicle Type
def get_car_make_model_type(soup, html_tag, html_class):
    full_titles = parseColumn(soup, html_tag, html_class)
    makes = []
    models = []
    vehicle_types = []
    valid_makes = ['BMW', 'Audi', 'Toyota', 'Lexus', 'Ford', 'Honda'
                   , 'Hyundai', 'Kia', 'Chevrolet', 'Jeep', 'Nissan', 'Volkswagen'
                   , 'Mitsubishi', 'Mazda', 'GMC', 'Cadillac', 'Land Rover', 'Dodge'           
                   , 'Subaru', 'Ram', 'Chrysler', 'Acura', 'Mercedes-Benz', 'Infiniti']
    
    valid_vehicle_type = ['Sedan', 'SUV', 'Coupe', 'Wagon']
    
    # Remove years
    for idx in range(len(full_titles)):
        full_titles[idx] = full_titles[idx][5:]
        
    for model in full_titles:
        clean_model = ''
        stripped_make = ''
        stripped_type = ''
        for make in valid_makes:
            if make.lower() in model.lower():
                stripped_make = make
                clean_model = model.replace(make, '')
        
        for vt in valid_vehicle_type:
            if vt.lower() in clean_model.lower():
                stripped_type = vt
                clean_model = clean_model.replace(vt, '')
                
        if (stripped_make != ''):
            makes.append(stripped_make.strip())
        else:
            makes.append(None) # if none matches add a none
        
        if (stripped_type != ''):
            vehicle_types.append(stripped_type.strip())
        else:
            vehicle_types.append(None) # if none matches add a none
        
        models.append(clean_model.strip()) 

    return makes, models, vehicle_types

# def get_numeric_vehicle_data(html_tag, html_class):
#     scraped_mileages = parseColumn(html_tag, html_class)
#     mileages = []
    
#     for mileage in scraped_mileages:
#         mileages.append(int(re.sub("[^0-9]", "", mileage)))
    
#     return mileages

def get_numeric_vehicle_data(soup, html_tag, html_class):
    scraped_data = parseColumn(soup, html_tag, html_class)
    parsed_data = []
    numeric_data = []
    
    for el in scraped_data:
        parsed_data.append(re.sub("[^0-9]", "", el))
        
    for data in parsed_data:
        if data == '':
            numeric_data.append(np.nan)
        else:
            numeric_data.append(int(data))
    
    return numeric_data

In [4]:
title = parseColumn('h2', 'ebiz-vdp-title color m-0')
title

['2014 Volkswagen Jetta',
 '2014 Volkswagen Passat',
 '2015 Nissan Rogue',
 '2019 Chevrolet Spark',
 '2016 Nissan Rogue',
 '2017 Mitsubishi Outlander',
 '2018 Kia Sportage',
 '2014 Chevrolet Camaro',
 '2020 Kia Soul',
 '2017 Honda Accord',
 '2018 Nissan Kicks',
 '2018 Toyota Corolla',
 '2016 Chevrolet Silverado 1500',
 '2019 Nissan Kicks',
 '2014 Chrysler 300C',
 '2019 Honda Civic',
 '2019 Ford Fusion Hybrid',
 '2018 Nissan Rogue',
 '2018 Chevrolet Trax',
 '2020 Mitsubishi Outlander Sport',
 '2018 Nissan Rogue Sport',
 '2016 Mercedes-Benz CLA',
 '2021 Toyota Corolla',
 '2015 Mercedes-Benz GLK',
 '2020 Mitsubishi Outlander',
 '2014 Ram 1500',
 '2019 Chevrolet Equinox',
 '2019 Jeep Compass',
 '2018 Ford Escape',
 '2019 Ford Escape',
 '2021 Nissan Sentra',
 '2014 Chevrolet Silverado 1500',
 '2021 Kia K5',
 '2020 Honda HR-V',
 '2018 Jeep Compass',
 '2018 Nissan Rogue',
 '2020 Mazda CX-5',
 '2020 Hyundai Santa Fe',
 '2014 Chevrolet Silverado 1500',
 '2014 GMC Sierra 1500',
 '2019 Chevrolet 

In [66]:
import numpy as np
def get_numeric_vehicle_data(html_tag, html_class):
    scraped_data = parseColumn(html_tag, html_class)
    parsed_data = []
    numeric_data = []
    
    for el in scraped_data:
        parsed_data.append(re.sub("[^0-9]", "", el))
        
    for data in parsed_data:
        if data == '':
            numeric_data.append(np.nan)
        else:
            numeric_data.append(int(data))
    
    return numeric_data

x = get_numeric_vehicle_data('div', 'price-item')
x

[10000,
 13497,
 17597,
 17997,
 18497,
 18997,
 20000,
 20000,
 20000,
 20597,
 20997,
 20997,
 21697,
 21997,
 21997,
 22997,
 23997,
 23997,
 23997,
 23997,
 24597,
 25000,
 25000,
 25000,
 25597,
 25697,
 25997,
 25997,
 25997,
 25997,
 26997,
 27497,
 27597,
 27697,
 27697,
 27797,
 27997,
 27997,
 28497,
 28997,
 28997,
 28997,
 28997,
 28997,
 nan,
 29197,
 nan,
 29697,
 nan,
 30000,
 30000,
 30000,
 30000,
 30197,
 30597,
 30697,
 30797,
 30897,
 30897,
 30997,
 30997,
 nan,
 31997,
 32997,
 33597,
 33597,
 33597,
 33997,
 33997,
 35000,
 35000,
 35000,
 35000,
 35000,
 35697,
 35997,
 35997,
 36597,
 36997,
 37497,
 37497,
 37997,
 37997,
 38597,
 38597,
 38997,
 38997,
 40000,
 40000,
 40000,
 40497,
 40497,
 40597,
 40897,
 40997,
 40997,
 41597,
 41697,
 nan,
 nan]

In [60]:
def get_numeric_vehicle_data(html_tag, html_class):
    parsed_data = parseColumn(html_tag, html_class)
    numeric_data = []
    
    for el in parsed_data:
        parsed_string = re.sub("[^0-9]", "", el)
        for data in parsed_string:
            if data === '':
                numeric_data.append(np.nan)
            else:
                numeric_data.append(int(parsed_string))
    
    return numeric_data



x = parseColumn('div', 'price-item')
for el in x:
    print(re.sub("[^0-9]", "", el))

10000
13497
17597
17997
18497
18997
20000
20000
20000
20597
20997
20997
21697
21997
21997
22997
23997
23997
23997
23997
24597
25000
25000
25000
25597
25697
25997
25997
25997
25997
26997
27497
27597
27697
27697
27797
27997
27997
28497
28997
28997
28997
28997
28997

29197

29697

30000
30000
30000
30000
30197
30597
30697
30797
30897
30897
30997
30997

31997
32997
33597
33597
33597
33997
33997
35000
35000
35000
35000
35000
35697
35997
35997
36597
36997
37497
37497
37997
37997
38597
38597
38997
38997
40000
40000
40000
40497
40497
40597
40897
40997
40997
41597
41697




In [58]:
prices2 = parseColumn('div', 'price-item')
prices2 = pd.Series(prices2).str.replace('\n', '')
prices2 = pd.Series(prices2).str.replace('Internet Price', '')

['\nInternet Price\n$10,000\n',
 '\nInternet Price\n$13,497\n',
 '\nInternet Price\n$17,597\n',
 '\nInternet Price\n$17,997\n',
 '\nInternet Price\n$18,497\n',
 '\nInternet Price\n$18,997\n',
 '\nInternet Price\n$20,000\n',
 '\nInternet Price\n$20,000\n',
 '\nInternet Price\n$20,000\n',
 '\nInternet Price\n$20,597\n',
 '\nInternet Price\n$20,997\n',
 '\nInternet Price\n$20,997\n',
 '\nInternet Price\n$21,697\n',
 '\nInternet Price\n$21,997\n',
 '\nInternet Price\n$21,997\n',
 '\nInternet Price\n$22,997\n',
 '\nInternet Price\n$23,997\n',
 '\nInternet Price\n$23,997\n',
 '\nInternet Price\n$23,997\n',
 '\nInternet Price\n$23,997\n',
 '\nInternet Price\n$24,597\n',
 '\nInternet Price\n$25,000\n',
 '\nInternet Price\n$25,000\n',
 '\nInternet Price\n$25,000\n',
 '\nInternet Price\n$25,597\n',
 '\nInternet Price\n$25,697\n',
 '\nInternet Price\n$25,997\n',
 '\nInternet Price\n$25,997\n',
 '\nInternet Price\n$25,997\n',
 '\nInternet Price\n$25,997\n',
 '\nInternet Price\n$26,997\n',
 '\nInte

In [5]:
# Prices
prices = parseColumn('h4', 'money-sign-disp body-font d-inline m-0')
prices

['$10,000',
 '$13,497',
 '$17,597',
 '$17,997',
 '$18,497',
 '$18,997',
 '$20,000',
 '$20,000',
 '$20,000',
 '$20,597',
 '$20,997',
 '$20,997',
 '$21,697',
 '$21,997',
 '$21,997',
 '$22,997',
 '$23,997',
 '$23,997',
 '$23,997',
 '$23,997',
 '$24,597',
 '$25,000',
 '$25,000',
 '$25,000',
 '$25,597',
 '$25,697',
 '$25,997',
 '$25,997',
 '$25,997',
 '$25,997',
 '$26,997',
 '$27,497',
 '$27,597',
 '$27,697',
 '$27,697',
 '$27,797',
 '$27,997',
 '$27,997',
 '$28,497',
 '$28,997',
 '$28,997',
 '$28,997',
 '$28,997',
 '$28,997',
 '$29,197',
 '$29,697',
 '$30,000',
 '$30,000',
 '$30,000',
 '$30,000',
 '$30,197',
 '$30,597',
 '$30,697',
 '$30,797',
 '$30,897',
 '$30,897',
 '$30,997',
 '$30,997',
 '$31,997',
 '$32,997',
 '$33,597',
 '$33,597',
 '$33,597',
 '$33,997',
 '$33,997',
 '$35,000',
 '$35,000',
 '$35,000',
 '$35,000',
 '$35,000',
 '$35,697',
 '$35,997',
 '$35,997',
 '$36,597',
 '$36,997',
 '$37,497',
 '$37,497',
 '$37,997',
 '$37,997',
 '$38,597',
 '$38,597',
 '$38,997',
 '$38,997',
 '$4

In [6]:
# Miles
miles = parseColumn('li', 'mileage-units')
miles

['Miles: 96,889',
 'Miles: 68,956',
 'Miles: 82,552',
 'Miles: 22,395',
 'Miles: 84,059',
 'Miles: 84,265',
 'Miles: 75,969',
 'Miles: 92,997',
 'Miles: 50,265',
 'Miles: 85,621',
 'Miles: 28,153',
 'Miles: 31,876',
 'Miles: 76,139',
 'Miles: 27,783',
 'Miles: 57,658',
 'Miles: 32,179',
 'Miles: 34,450',
 'Miles: 51,021',
 'Miles: 26,323',
 'Miles: 45,072',
 'Miles: 36,657',
 'Miles: 70,615',
 'Miles: 20,660',
 'Miles: 60,319',
 'Miles: 48,332',
 'Miles: 85,103',
 'Miles: 29,799',
 'Miles: 38,252',
 'Miles: 26,732',
 'Miles: 29,368',
 'Miles: 5,182',
 'Miles: 90,329',
 'Miles: 19,388',
 'Miles: 20,655',
 'Miles: 39,375',
 'Miles: 37,749',
 'Miles: 34,044',
 'Miles: 53,179',
 'Miles: 65,619',
 'Miles: 78,939',
 'Miles: 55,699',
 'Miles: 71,040',
 'Miles: 6,889',
 'Miles: 58,477',
 'Miles: 39,131',
 'Miles: 77,335',
 'Miles: 55,180',
 'Miles: 73,459',
 'Miles: 41,460',
 'Miles: 67,916',
 'Miles: 81,799',
 'Miles: 46,777',
 'Miles: 22,960',
 'Miles: 62,566',
 'Miles: 73,012',
 'Miles: 40,

In [7]:
# VIN
vin = parseColumn('li', 'vin')
vin

['VIN #: 3VW1K7AJ1EM371670',
 'VIN #: 1VWAT7A35EC091203',
 'VIN #: KNMAT2MV0FP507719',
 'VIN #: KL8CF6SA2KC750949',
 'VIN #: JN8AT2MV3GW141234',
 'VIN #: JA4AZ2A38HZ070328',
 'VIN #: KNDPMCAC4J7345148',
 'VIN #: 2G1FC1E33E9311597',
 'VIN #: KNDJ23AU7L7006276',
 'VIN #: 1HGCR2F12HA154683',
 'VIN #: 3N1CP5CU0JL509193',
 'VIN #: 2T1BURHE8JC083752',
 'VIN #: 1GCNCNEH9GZ311344',
 'VIN #: 3N1CP5CU8KL509315',
 'VIN #: 2C3CCAKT3EH203134',
 'VIN #: 2HGFC2F61KH520426',
 'VIN #: 3FA6P0LU5KR125847',
 'VIN #: 5N1AT2MV3JC764725',
 'VIN #: 3GNCJPSB4JL155115',
 'VIN #: JA4AR3AU1LU019354',
 'VIN #: JN1BJ1CR9JW289354',
 'VIN #: WDDSJ4GB7GN374614',
 'VIN #: JTDEPMAE4MJ129197',
 'VIN #: WDCGG8JBXFG350075',
 'VIN #: JA4AZ3A36LZ025421',
 'VIN #: 3C6JR7CT6EG330728',
 'VIN #: 3GNAXSEVXKL292124',
 'VIN #: 3C4NJDBB1KT647232',
 'VIN #: 1FMCU9HD5JUA38116',
 'VIN #: 1FMCU9GD3KUB66924',
 'VIN #: 3N1AB8DV0MY250548',
 'VIN #: 1GCVKREC7EZ306879',
 'VIN #: 5XXG14J21MG054513',
 'VIN #: 3CZRU6H16LM703125',
 'VIN #: 3C4NJ

In [8]:
# Transmission
transmission = parseColumn('ul', 'small list-unstyled mb-0')
transmission[0].split('\n')

for el in transmission[0].split('\n'):
    if 'Engine' in el:
        print(el.split(':')[1].strip())

2.0L 4 Cylinder


In [123]:
# Use for when vehicle data doesn't have specific class names
def get_vehicle_data_direct_mecca(soup, html_tag, html_class, vehicle_data_type):
    misc_vehicle_data = parseColumn(soup, html_tag, html_class)
    vehicle_data_list = []
    for row in misc_vehicle_data:
        for el in row.split('\n'):
            if vehicle_data_type.lower() in el.lower():
                vehicle_data_list.append(el.split(':')[1].strip())
    return vehicle_data_list
                
                
engine = get_vehicle_data_direct_mecca(soup, 'ul', 'small list-unstyled mb-0', 'engine')
engine

['2.0L 4 Cylinder',
 '1.8L 4 Cylinder',
 '2.5L 4 Cylinder',
 '1.4L 4 Cylinder',
 '2.5L 4 Cylinder',
 '2.4L 4 Cylinder',
 '2.4L 4 Cylinder',
 '3.6L V6 Cylinder',
 '2.0L 4 Cylinder',
 '2.4L 4 Cylinder',
 '1.6L 4 Cylinder',
 '1.8L 4 Cylinder',
 '4.3L V6 CYLINDER',
 '1.6L 4 Cylinder',
 '5.7L 8 Cylinder',
 '2.0L 4 Cylinder',
 '2.0L 4 Cylinder',
 '2.5L 4 Cylinder',
 '1.4L 4 CYLINDER',
 '2.0L 4 Cylinder',
 '2.0L 4 Cylinder',
 '2.0L 4 Cylinder',
 '1.8L 4 Cylinder',
 '3.5L V6 Cylinder',
 '2.4L 4 Cylinder',
 '5.7L 8 Cylinder',
 '1.5L 4 Cylinder',
 '2.4L 4 Cylinder',
 '1.5L 4 Cylinder',
 '1.5L 4 Cylinder',
 '2.0L 4 Cylinder',
 '2.4L 4 Cylinder',
 '5.3L 8 Cylinder',
 '1.6L 4 Cylinder',
 '1.8L 4 Cylinder',
 '2.4L 4 Cylinder',
 '2.5L 4 Cylinder',
 '2.5L 4 CYLINDER',
 '2.4L 4 Cylinder',
 '4.3L V6 Cylinder',
 '5.3L 8 Cylinder',
 '3.6L V6 Cylinder',
 '4.3L V6 Cylinder',
 '2.4L 4 CYLINDER',
 '3.6L V6 Cylinder',
 '3.6L V6 Cylinder',
 '5.7L 8 Cylinder',
 '2.0L 4 Cylinder',
 '3.6L V6 Cylinder',
 '2.0L 4 Cy

In [ ]:
dealerships = {
    'Bostonyan Auto Group': {
        'url': 'https://www.bostonyanautogroup.com/view-inventory',
        'dealership_name': 'Bostonyan Auto Group',
        'address': '119 Worcester St',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    },
    'Direct Auto Mecca': {
        'url': 'https://www.directautomecca.com/view-inventory.aspx',
        'dealership_name': 'Direct Auto Mecca',
        'address': '154 Waverly Street',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    }
}

In [129]:
# Scrape inventory HTML
def get_direct_auto_inventory_data(soup, dealership_info):    
    # Initialize empty data frame
    cars = pd.DataFrame()
    
    # Add title
    title = parseColumn(soup, 'h2', 'ebiz-vdp-title color m-0')    
    addColumnDF(cars, title, 'title')

    # Add vehicle manufacture date
    years = get_valid_year(soup, 'h2', 'ebiz-vdp-title color m-0')
    addColumnDF(cars, years, 'year')

    # Add make, models, and vehicle type
    makes, models, vtypes = get_car_make_model_type(soup, 'h2', 'ebiz-vdp-title color m-0')
    addColumnDF(cars, makes, 'make')
    addColumnDF(cars, models, 'models')
    addColumnDF(cars, vtypes, 'vehicle_type')

    # Add mileage col
    miles = get_numeric_vehicle_data(soup, 'li', 'mileage-units')
    addColumnDF(cars, miles, 'vehicle_mileage')

    # Add price
    car_prices = get_numeric_vehicle_data(soup, 'div', 'price-item')
    print(car_prices)
    addColumnDF(cars, car_prices, 'price')

    # Add Colors & transmission & other cols
    addColumnDF(cars, get_vehicle_data_direct_mecca(soup, 'ul', 'small list-unstyled mb-0', 'exterior'), 'exterior_color')
    addColumnDF(cars, get_vehicle_data_direct_mecca(soup, 'ul', 'small list-unstyled mb-0', 'interior'), 'interior_color')
    addColumnDF(cars, get_vehicle_data_direct_mecca(soup, 'ul', 'small list-unstyled mb-0', 'transmission'), 'transmission')
    addColumnDF(cars, get_vehicle_data_direct_mecca(soup, 'ul', 'small list-unstyled mb-0', 'engine'), 'engine')
    addColumnDF(cars, parseColumn(soup, 'li', 'vin'), 'vin')
    cars['vin'] = cars['vin'].str.replace('VIN #: ', '')
    
    # Add dealership info + scrape date
    cars['dealership_name'] = dealership_info['dealership_name']
    cars['dealership_address'] = dealership_info['address']
    cars['dealership_zipcode'] = dealership_info['zipcode']
    cars['dealership_city'] = dealership_info['city']
    cars['dealership_state'] = dealership_info['state']
    cars['inventory_url'] = dealership_info['url']
    cars['scraped_date'] = datetime.now(tz = None)
    
    return cars
    # Add data to a SQLite database
#     conn = sqlite3.connect('cars.db')
#     cars.to_sql('inventory', conn, if_exists='append', index=False)
    

In [130]:
response = requests.get(dealerships['Direct Auto Mecca']['url'], headers = headers)
soup = BeautifulSoup(response.text, "html.parser")  
c = get_direct_auto_inventory_data(soup, dealerships['Direct Auto Mecca'])

[10000, 13497, 17597, 17997, 18497, 18997, 20000, 20000, 20000, 20597, 20997, 20997, 21697, 21997, 21997, 22997, 23997, 23997, 23997, 23997, 24597, 25000, 25000, 25000, 25597, 25697, 25997, 25997, 25997, 25997, 26997, 26997, 27497, 27597, 27697, 27697, 27797, 27997, 27997, 28497, 28997, 28997, 28997, 28997, 28997, nan, 29197, nan, 29697, nan, 30000, 30000, 30000, 30000, 30197, 30597, 30697, 30797, 30897, 30897, 30997, 30997, nan, 31997, 32997, 33597, 33597, 33597, 33997, 33997, 35000, 35000, 35000, 35000, 35000, 35697, 35997, 35997, 36597, 36997, 37497, 37497, 37997, 37997, 38597, 38597, 38997, 38997, 40000, 40000, 40000, 40497, 40497, 40597, 40897, 40997, 40997, 41597, 41697, nan]


In [131]:
c.isnull().sum()

title                   0
year                    0
make                    0
models                  0
vehicle_type          100
vehicle_mileage         0
price                   5
exterior_color          0
interior_color          0
transmission            0
engine                  0
vin                     0
dealership_name         0
dealership_address      0
dealership_zipcode      0
dealership_city         0
dealership_state        0
inventory_url           0
scraped_date            0
dtype: int64

In [132]:
c.loc[c['price'].isnull(), ['title', 'price', 'vin']]

,title,price,vin
45,2017 Chevrolet Camaro,NaN,1G1FB3DS9H0197305
47,2020 Ford Escape,NaN,1FMCU9J98LUB19944
49,2018 Subaru WRX,NaN,JF1VA1C63J9806104
62,2016 Ford F-150,NaN,1FTFW1EF7GFD27806
99,2018 Ford Transit-250,NaN,1FTYR2YM8JKB09923


In [133]:
c

,title,year,make,models,vehicle_type,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2014 Volkswagen Jetta,2014,Volkswagen,Jetta,None,96889,10000.0,Silver,Mar Grigalauskas,5 Spd Manual,2.0L 4 Cylinder,3VW1K7AJ1EM371670,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
1,2014 Volkswagen Passat,2014,Volkswagen,Passat,None,68956,13497.0,Silver,Gray,6 Spd Automatic,1.8L 4 Cylinder,1VWAT7A35EC091203,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
2,2015 Nissan Rogue,2015,Nissan,Rogue,None,82552,17597.0,Super Black,Black,CVT,2.5L 4 Cylinder,KNMAT2MV0FP507719,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
3,2019 Chevrolet Spark,2019,Chevrolet,Spark,None,22395,17997.0,Silver Ice Metallic,Jet Black/Dark Anderson Silver Metallic,CVT,1.4L 4 Cylinder,KL8CF6SA2KC750949,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
4,2016 Nissan Rogue,2016,Nissan,Rogue,None,84059,18497.0,Gun Metallic,Charcoal,CVT,2.5L 4 Cylinder,JN8AT2MV3GW141234,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018 Ford Transit-250,2018,Ford,Transit-250,None,39113,40997.0,White,Black,6 Spd Automatic,3.7L V6 Cylinder,1FTYR1ZM2JKB48001,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
96,2019 Ram ProMaster 2500,2019,Ram,ProMaster 2500,None,59259,40997.0,Bright White Clearcoat,Black,6 Spd Automatic,3.6L V6 Cylinder,3C6TRVCG7KE539175,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
97,2018 Ram ProMaster 2500,2018,Ram,ProMaster 2500,None,53371,41597.0,Bright White Clearcoat,Black,6 Spd Automatic,3.6L V6 Cylinder,3C6TRVCG6JE112911,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842
98,2018 Ram ProMaster 2500,2018,Ram,ProMaster 2500,None,53055,41697.0,Bright White Clearcoat,Black,6 Spd Automatic,3.6L V6 Cylinder,3C6TRVCGXJE138699,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:33:33.293842


In [88]:
# Scrape inventory HTML
response = requests.get('https://www.directautomecca.com/inventory.aspx?_new=true&_used=true&_page=3', headers = headers)
soup_test = BeautifulSoup(response.text, "html.parser")   

In [92]:
soup_test

<!DOCTYPE html>

<html dir="ltr" lang="en-US">
<head>
<title>New, Used Cars at DirectAutomecca.com Serving Framingham, MA, Inventory</title>
<meta content="Serving Framingham, Massachusetts (MA), DirectAutomecca.com is the place to purchase your next car. View photos and details of our entire inventory." name="description"/>
<meta content="DirectAutomecca.com Serving Framingham, MA, Inventory, New, Used" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<meta content="DirectAutomecca.com" name="author">
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="New, Used Cars at DirectAutomecca.com Serving Framingham, MA, Inventory" name="og:title"/>
<meta content="website" name="og:type"/>
<meta content="DirectAutomecca.com" property="og:site_name"/>
<meta content="http://www.directautomecca.com/inventory.aspx?_new=true&amp;_used=true&amp;_page=3" name="og:url"/>
<meta 

In [91]:
def parseColumn(soup, html_tag, html_class):
    dataColumn = soup.find_all(html_tag, class_=html_class)
    new_col_list = []
    
    for i in range(0, len(dataColumn)):
        new_col_list.append(dataColumn[i].get_text())
        
    return new_col_list

title = parseColumn(soup_test, 'h2', 'ebiz-vdp-title color m-0')
title

[]

In [99]:
dealerships = {
    'Bostonyan Auto Group': {
        'url': 'https://www.bostonyanautogroup.com/view-inventory',
        'pagination_url': '',
        'dealership_name': 'Bostonyan Auto Group',
        'address': '119 Worcester St',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    },
    'Direct Auto Mecca': {
        'url': 'https://www.directautomecca.com/view-inventory.aspx',
        'pagination_url': 'https://www.directautomecca.com/inventory.aspx?_new=true&_used=true&_page=2',
        'dealership_name': 'Direct Auto Mecca',
        'address': '154 Waverly Street',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    }
}

In [139]:
pagination_url = dealerships['Direct Auto Mecca']['pagination_url']
page_counter = 2

blah = []

while (True):
    response = requests.get(pagination_url, headers = headers)
    soup = BeautifulSoup(response.text, "html.parser")   
    title = parseColumn(soup, 'h2', 'ebiz-vdp-title color m-0')
    
    if len(title) == 0:
        break;
    else:
        blah.append(get_direct_auto_inventory_data(soup, dealerships[key]))
    
    page_counter += 1
    pagination_url = pagination_url.replace('page=2', f'page={page_counter}')   
    

    


[nan, 41997, 42397, 42997, 42997, 42997, 43997, 43997, 43997, 44497, 44697, 45000, 45000, 45000, 45000, 45000, 45000, 45597, 45997, 47597, 47697, 47997, 47997, 48697, 48997, 48997, 49497, 49497, 49497, nan, 50000, 50000, 50997, 50997, 51997, 52997, 52997, 52997, 52997, 53997, 55497, nan, 56997, 57997, 57997, 59497, 59497, 59497, 60000, 60997, 65000, 65000]


In [141]:
blah

[                              title  year           make            models  \
 0       2019 Chevrolet Express 2500  2019      Chevrolet      Express 2500   
 1                   2020 Ford F-150  2020           Ford             F-150   
 2                2019 Tesla Model 3  2019           None                     
 3              2020 GMC Savana 2500  2020            GMC       Savana 2500   
 4                    2015 GMC Yukon  2015            GMC             Yukon   
 5                     2019 Ram 1500  2019            Ram              1500   
 6     2019 Chevrolet Silverado 1500  2019      Chevrolet    Silverado 1500   
 7                2018 Toyota Tundra  2018         Toyota            Tundra   
 8       2019 Chevrolet Express 3500  2019      Chevrolet      Express 3500   
 9     2019 Chevrolet Silverado 1500  2019      Chevrolet    Silverado 1500   
 10             2019 GMC Sierra 1500  2019            GMC       Sierra 1500   
 11               2020 Ford Explorer  2020          

In [111]:
pagination_url = dealerships['Direct Auto Mecca']['pagination_url']
page_counter = 2

titles = []

response = requests.get(pagination_url, headers = headers)
soup = BeautifulSoup(response.text, "html.parser")   
title = parseColumn(soup, 'h2', 'ebiz-vdp-title color m-0')

titles.append(title)

page_counter += 1
pagination_url = pagination_url.replace('page=2', f'page={page_counter}')    



In [112]:
titles

[['2019 Chevrolet Express 2500',
  '2020 Ford F-150',
  '2019 Tesla Model 3',
  '2020 GMC Savana 2500',
  '2015 GMC Yukon',
  '2019 Ram 1500',
  '2019 Chevrolet Silverado 1500',
  '2018 Toyota Tundra',
  '2019 Chevrolet Express 3500',
  '2019 Chevrolet Silverado 1500',
  '2019 GMC Sierra 1500',
  '2020 Ford Explorer',
  '2020 Ford Explorer',
  '2019 Ram 1500',
  '2020 Chevrolet Express 2500',
  '2020 Chevrolet Express 2500',
  '2019 Ford Transit-250',
  '2020 GMC Sierra 1500',
  '2020 Ford Transit-250',
  '2017 Mercedes-Benz GLC',
  '2019 Dodge Durango',
  '2020 Chevrolet Silverado 1500',
  '2019 Ram ProMaster 2500',
  '2020 Ford Transit-250',
  '2019 Toyota Tundra',
  '2018 Ford F-150',
  '2020 Chevrolet Silverado 1500',
  '2020 Chevrolet Silverado 1500',
  '2020 Ram 1500',
  '2020 Chevrolet Express 2500',
  '2021 Ford F-150',
  '2017 GMC Sierra 3500HD',
  '2021 Ram 2500',
  '2019 Ford F-250SD',
  '2019 Ford F-150',
  '2019 Tesla Model 3',
  '2019 Tesla Model 3',
  '2020 Ford Explorer

In [80]:
# conn = sqlite3.connect('cars.db')

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Doping EMPLOYEE table if already exists
# cursor.execute("DROP TABLE inventory")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()

In [ ]:
# Check pagination

https://www.directautomecca.com/inventory.aspx?_new=true&_used=true&_page=2

In [142]:
conn = sqlite3.connect('cars.db')

In [143]:
df = pd.read_sql_query("SELECT * from inventory", conn)
df

,title,year,make,models,vehicle_type,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2009 Toyota Camry LE Sedan,2009,Toyota,Camry LE,Sedan,124351,8495.0,Magnetic Gray Metallic,Ash,Automatic,2.4L I4 158hp 161ft. lbs.,4T4BE46K89R114118,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-04 23:43:26.413657
1,2011 BMW X3 xDrive28i SUV,2011,BMW,X3 xDrive28i,SUV,98952,13995.0,Jet Black,Black Nevada Leather,Automatic,3.0L I6 240hp 221ft. lbs.,5UXWX5C51BL715699,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-04 23:43:26.413657
2,2011 Lexus HS 250h Sedan,2011,Lexus,HS 250h,Sedan,92948,13995.0,Black,Black,Automatic,2.4L Hybrid I4 147hp 138ft. lbs.,JTHBB1BA0B2044987,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-04 23:43:26.413657
3,2011 Mercedes-Benz ML 350 4MATIC SUV,2011,Mercedes-Benz,ML 350 4MATIC,SUV,98922,14995.0,Obsidian Black Metallic,Black Leather,Automatic,3.5L V6 268hp 258ft. lbs.,4JGBB8GB4BA723044,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-04 23:43:26.413657
4,2012 Audi A6 3.0T quattro Prestige Sedan,2012,Audi,A6 3.0T quattro Prestige,Sedan,98933,17995.0,White,Brown,Automatic,3.0L Supercharged V6 310hp 325ft. lbs.,WAUHGAFC4CN137257,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-04 23:43:26.413657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2019 Ram 1500,2019,Ram,1500,None,30222,59497.0,Granite Crystal Metallic Clearcoat,Black,8 Spd Automatic,5.7L 8 Cylinder,1C6SRFHT3KN725878,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
169,2020 Chevrolet Silverado 2500HD,2020,Chevrolet,Silverado 2500HD,None,16111,60000.0,Black,Jet Black,6 Spd Automatic,6.0L 8 Cylinder,1GC1YME77LF226516,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
170,2021 Chevrolet Silverado 2500HD,2021,Chevrolet,Silverado 2500HD,None,11178,60997.0,White,Black,6 Spd Automatic,6.6L 8 Cylinder,1GC4YME72MF192760,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
171,2020 GMC Sierra 1500,2020,GMC,Sierra 1500,None,20840,65000.0,White,Black,10 Spd Automatic,6.2L 8 Cylinder,1GTP9EEL5LZ144700,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412


In [146]:
df.loc[df['dealership_name'] == 'Direct Auto Mecca', :]

,title,year,make,models,vehicle_type,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
21,2014 Volkswagen Jetta,2014,Volkswagen,Jetta,None,96889,10000.0,Silver,Mar Grigalauskas,5 Spd Manual,2.0L 4 Cylinder,3VW1K7AJ1EM371670,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:29.300674
22,2014 Volkswagen Passat,2014,Volkswagen,Passat,None,68956,13497.0,Silver,Gray,6 Spd Automatic,1.8L 4 Cylinder,1VWAT7A35EC091203,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:29.300674
23,2015 Nissan Rogue,2015,Nissan,Rogue,None,82552,17597.0,Super Black,Black,CVT,2.5L 4 Cylinder,KNMAT2MV0FP507719,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:29.300674
24,2019 Chevrolet Spark,2019,Chevrolet,Spark,None,22395,17997.0,Silver Ice Metallic,Jet Black/Dark Anderson Silver Metallic,CVT,1.4L 4 Cylinder,KL8CF6SA2KC750949,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:29.300674
25,2016 Nissan Rogue,2016,Nissan,Rogue,None,84059,18497.0,Gun Metallic,Charcoal,CVT,2.5L 4 Cylinder,JN8AT2MV3GW141234,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:29.300674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2019 Ram 1500,2019,Ram,1500,None,30222,59497.0,Granite Crystal Metallic Clearcoat,Black,8 Spd Automatic,5.7L 8 Cylinder,1C6SRFHT3KN725878,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
169,2020 Chevrolet Silverado 2500HD,2020,Chevrolet,Silverado 2500HD,None,16111,60000.0,Black,Jet Black,6 Spd Automatic,6.0L 8 Cylinder,1GC1YME77LF226516,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
170,2021 Chevrolet Silverado 2500HD,2021,Chevrolet,Silverado 2500HD,None,11178,60997.0,White,Black,6 Spd Automatic,6.6L 8 Cylinder,1GC4YME72MF192760,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
171,2020 GMC Sierra 1500,2020,GMC,Sierra 1500,None,20840,65000.0,White,Black,10 Spd Automatic,6.2L 8 Cylinder,1GTP9EEL5LZ144700,Direct Auto Mecca,154 Waverly Street,01760,Natick,MA,https://www.directautomecca.com/view-inventory...,2022-03-04 23:43:32.375412
